In [ ]:

import meep as mp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video
from matplotlib.patches import Polygon
import pandas as pd


In [ ]:


from datetime import datetime

now=datetime.now()
thetime=str(now.day)+str(now.hour)+str(now.minute)
abspath='/mnt/d/Dai(COMSOL2)/meepmedia/plots/'

In [ ]:
a = 1
r=0.24
n=12
theta=0
Bar = 8 # the barrier PhC to confine light
Ls=[20,20]# a list of the length of bent wg segments
parity=np.power(-1,len(Ls))
output_index=int((parity+1)/2) #the final waveguide may direct downwards or rightwards
Ls_temp=Ls.copy()

Pso=5
Wso=5
Pin=10
Pout=5
Wport=10

extra_N=np.abs(Pso)+3 # add some extra length and then delete it, so as to smooth the edge
Ls_temp[0]+=extra_N
Ls_temp[-1]+=extra_N


In [ ]:


def data(Lst):  # split the x and y data stored in a nested list, thus ready for pyplot line plotting
    k = 0
    xxx = []
    yyy = []

    while k < len(Lst):
        xxx.append(Lst[k][0])
        yyy.append(Lst[k][1])

        k = k + 1
    return xxx, yyy


def distance(v1, v2):  # distance between two 2d points
    d = np.sqrt((v1[0] - v2[0]) ** 2 + (v1[1] - v2[1]) ** 2)
    return d

def moveframe(original_frame, vector):
    new_frame = []
    for var in original_frame:
        new_frame.append([var[0] + vector[0], var[1] + vector[1]])
    return new_frame

def frameX(N0, M0):
    frame = []
    for i, j in np.ndindex((N0, M0)):
        frame.append([a * i, a * j])
    return frame

def frameM(N0, M0):  # the frame
    frame = []
    for i, j in np.ndindex((N0, M0)):
        frame.append([a*np.sqrt(2) * (i+0.25)-N0*np.sqrt(2)*a/2, a*np.sqrt(2) * (j+0.25)-M0*np.sqrt(2)*a/2])
        frame.append([a*np.sqrt(2) * (i+0.75)-N0*np.sqrt(2)*a/2, a*np.sqrt(2) * (j+0.75)-M0*np.sqrt(2)*a/2])

    return frame


def polygons(r0, n0, theta0):
    polygon = []
    for i in range(n0):
        polygon.append(
            [r0 * np.sin(2 * np.pi / n0 * i + theta0 / 180 * np.pi),
             r0 * np.cos(2 * np.pi / n0 * i + theta0 / 180 * np.pi)])
    return polygon


def patterning(Frame, Polygon):  # polygons have default center (0,0). create polygon arrays on the frame.
    bulk = []
    for i in range(len(Frame)):
        on_center = Frame[i]
        New_poly = []
        for j in range(len(Polygon)):
            vertice = [sum(x) for x in zip(on_center, Polygon[j])]
            New_poly.append(vertice)
            
        bulk.append(New_poly)
    return bulk

def meepvector(point):
    result=mp.Vector3(point[0],point[1])

    return result

def meepshape(shape):
    meep_shape=[]
    for vertice in shape:
        meep_shape.append(meepvector(vertice))

    return meep_shape

def meepprism(shape,material):
    result=mp.Prism(vertices=meepshape(shape),height=mp.inf,material=material)

    return result

def meepbulk(bulk,material):
    geo=[]
    for shape in bulk:
        geo.append(meepprism(shape,material))

    return  geo


def parameter_generator(List_seg):
    global parity
    Neven = sum(List_seg[::2])
    Nodd = sum(List_seg[1::2])
    Mb=int(Nodd+Bar+(1-parity)/2*Bar)
    Nb=int(Neven+(parity+1)/2*Bar)
    Ns = sum(List_seg)
    Ms = Bar * 2
    spine_s = [[-Ns*np.sqrt(2)*a/2, 0], [Ns * a*np.sqrt(2)/2, 0]]
    spine_b = [[-Nb*np.sqrt(2)*a/2, (Mb/2-Bar) * a*np.sqrt(2)]]
    for i in range(len(List_seg)):
        parity1 = np.power(-1, i)
        new_point = spine_b[-1][:]
        new_point[int((1 - parity1) / 2)] += parity1 * List_seg[i]*a*np.sqrt(2)
        spine_b.append(new_point)
    List_s=[Ns, Ms, spine_s]
    List_b=[Nb, Mb, spine_b]
    package=[List_s,List_b]

    return package


def outside(obj,spine):
    value=1
    for point in spine:
        if obj[0]<point[0] and obj[1]<point[1]:
            value*=0
        else:
            value*=1
    return value==0


def cutblokcs(frame,spine):
    upper=[]
    lower=[]
    for point in frame:
        if outside(point,spine):
            lower.append(point)

        else:
            upper.append(point)
    return upper, lower


def cleanedge(frame,spine):
    new_frame=[]
    left=spine[0][0]+extra_N*a*np.sqrt(2)

    if len(spine)>2:
        right=spine[-1][int((parity+1)/2)]+parity*extra_N*a*np.sqrt(2)
        for point in frame:
            if point[0]>=left and (right-point[int((parity+1)/2)])*parity<=0:
                new_frame.append(point)

    else:
        right=spine[1][0]-extra_N*a*np.sqrt(2)
        for point in frame:
            if left<= point[0]<=right:
                new_frame.append(point)
    return new_frame



# try to plot a bent waveguide
def makebent(S):

    global Ls,Ls_temp
    polygon=polygons(r,n,theta)
    param_package=parameter_generator(Ls_temp)
    spines=[param_package[0][2],param_package[1][2]]
    bulk_package=[]
    for i in [0,1]:
        list_sb=param_package[i]
        NN,MM,sspine=list_sb
        fframe=frameM(NN,MM)
        upper, lower = cutblokcs(fframe, sspine)
        upper = moveframe(upper, (S * a / np.sqrt(2)/2, S * a / np.sqrt(2)/2))
        lower = moveframe(lower, (-S * a / np.sqrt(2)/2, -S * a / np.sqrt(2)/2))
        #print("origionallower=",lower[0:3])
        upper=cleanedge(upper,sspine)
        lower=cleanedge(lower,sspine)
        bulk_upper = patterning(upper, polygon)
        bulk_lower = patterning(lower, polygon)
        #print("lower=",lower[0:3])
        bulk_package.append([bulk_upper,bulk_lower])

    return bulk_package,param_package

In [ ]:
decay="i"   # i for running till intensity decays to [check_dBm], t for fixed sim time, g for quick geometry check, a for exporting animation 
ani=bool(decay=="a")
Tvideo=300  # decay time used for making animation
n_eff = 2.7
f_input=225
check_dBm = 30
a0=500
c_const0=299792458
fcen=1000*f_input*a0/c_const0  #normalized center freq
df = 0.1
nfreq = 500 # number of frequencies at which to compute flux
dpml=1
parameterS=2


resolution = 16
eps=n_eff**2
eps2=1  # sometimes the input wire is not needed, set its eps to 1.

In [ ]:

def phc_trans(wgtype=0, T_decay=1000):   #wgtype=0 for straight wg, 1 for bent wg, 0.1 for free space but skips the animation code, 0.2 for free space as well but can output mode evolution.mp4

    bulk_package,params=makebent(parameterS)
    print("all is fine here")

    def meepports():
    
        port_package=[]
        for i in [0,1]:
            spine=params[i][2]
            left=spine[0][0]+extra_N*np.sqrt(2)*a
            source=[mp.Source(mp.GaussianSource(fcen, fwidth=df),
                              component=mp.Ez,
                              center=mp.Vector3(left-Pso*np.sqrt(2),spine[0][1]),
                              size=mp.Vector3(0,Wso))]

            port1=mp.FluxRegion(center=mp.Vector3(left+Pin*np.sqrt(2),spine[0][1]),size=mp.Vector3(0,Wport))
            decay_check=mp.Vector3(spine[-1][0]+(extra_N+Pin)*np.sqrt(2)*(((1-output_index)*parity)*i+(i-1)),spine[-1][1]+i*(extra_N+Pin)*np.sqrt(2)*(output_index)*parity)
            port2=mp.FluxRegion(center=decay_check,size=mp.Vector3(Wport*output_index*i,Wport*(1-i*output_index)))

            ports_by_wg=[source,port1,port2,decay_check]
            port_package.append(ports_by_wg)  #index 0 fr str, index 1 for bent
        return port_package

    def sizes():
        return [params[0][i]*a*np.sqrt(2) for i in [0,1]],[params[1][i]*a*np.sqrt(2) for i in [0,1]]


    cell=mp.Vector3(sizes()[int(wgtype)][0],sizes()[int(wgtype)][1],0)
    if wgtype in [0.1,0.2]:
        geometry = []   
    else:
        geometry=meepbulk(sum(bulk_package[int(wgtype)],[]),mp.Medium(epsilon=eps))

        #print(sum(bulk_package[int(wgtype)],[]))
    #waveguide = mp.Block(mp.Vector3(mp.inf,ww,mp.inf),center=mp.Vector3(0,0,0),material=mp.Medium(epsilon=eps))
    
    
    pml_layers = [mp.PML(dpml)]

    source,port1,port2,decay_check=meepports()[int(wgtype)]
    sim = mp.Simulation(cell_size=cell,
                        boundary_layers=pml_layers,
                        geometry=geometry,
                        sources=source,
                        resolution=resolution)

    fluxes=[sim.add_flux(fcen, df, nfreq, port) for port in [port1,port2]]

    if ani and wgtype in [0,1,0.2]:
        f = plt.figure(dpi=150)
        Animate = mp.Animate2D(sim, fields=mp.Ez, f=f,
                               #field_parameters={'alpha':0.8, 'cmap':'RdBu', 'interpolation':'none'},
                               realtime=False, normalize=True)

        sim.run(mp.at_every(0.25,Animate),until=Tvideo)

        plt.close()

        filename1 = thetime+"_S="+str(parameterS)+"_wg="+str(wgtype)+'_freq='+str(f_input)+'_T='+str(Tvideo)
        filename2 = '/mnt/d/Dai(COMSOL2)/meepmedia/video2/'+filename1
        filename = filename2+".mp4"
        fps = 60
        Animate.to_mp4(fps,filename)
        Video(filename)
    else:

        # show geometry
        %matplotlib inline
        f = plt.figure(dpi=150)
        sim.plot2D(ax=f.gca())

        filename=abspath+thetime+"wg="+str(wgtype)
        if decay in ["i","t"]:
            plt.savefig(filename+".jpg")
        plt.show()
        
        if decay=="i":
            sim.run(until_after_sources=mp.stop_when_fields_decayed(100, mp.Ez, decay_check, pow(10,-check_dBm/10)))
            #plt.savefig(filename+".jpg")
        elif decay=="t":
            sim.run(until=T_decay)
            #plt.savefig(filename+".jpg")
        elif decay=="g":
            sim.run(until=1)

    results = [mp.get_flux_freqs(fluxes[-1])]
    for var in fluxes:
        results.append(mp.get_fluxes(var))

    return results


In [ ]:
result_none=phc_trans(wgtype=0.2)  # only ports and free space

In [ ]:
result_str=phc_trans(wgtype=0,T_decay=2000)

In [ ]:
result_bent=phc_trans(wgtype=1,T_decay=3000)

In [ ]:

def sortports(result):
    result_array=np.array(result)
    f,p1,p2=result_array
    return f,p1,p2

abspath='/mnt/d/Dai(COMSOL2)/meepmedia/plots/'

In [ ]:

try:
    fig, ax = plt.subplots()
    f,p1_none,p2_none=sortports(result_none)
    freqs=c_const0 * np.array(f)/a0/1000
    ax.plot(freqs,p1_none,label="p1")
    ax.plot(freqs,p2_none,label="p2")
    plt.legend(fontsize=20)
    if decay in ["t","i"]:    
        plt.savefig(abspath+thetime+"none.png")
    #plt.ylim([0,1.2])
except Exception as e: print(e)

In [ ]:

try:
    fig, ax = plt.subplots()
    f,p1_str,p2_str=sortports(result_str)
    couplingE=p1_str/p1_none
    freqs=c_const0 * np.array(f)/a0/1000
    ax.plot(freqs,couplingE,label="Coupling Efficiency")
    #ax.plot(freqs,p2_str,label="p2")
    plt.legend(fontsize=20)
    if decay in ["t","i"]:    
        plt.savefig(abspath+thetime+"CE.png")
    #plt.ylim([0,1.2])
except Exception as e: print(e)

In [ ]:

try:
    fig, ax = plt.subplots()
    f,p1_bent,p2_bent=sortports(result_bent)
    if parity==1:
        p2_bent=[element*-1 for element in p2_bent]

    T_str=p2_str/p1_str
    T_bent=p2_bent/p1_str
    freqs=c_const0 * np.array(f)/a0/1000
    ax.plot(freqs,T_str,label="Str")
    ax.plot(freqs,T_bent,label="Bent")
    plt.legend(fontsize=20)
    plt.ylim([0,1.2])
    if decay in ["t","i"]:    
        plt.savefig(abspath+thetime+"_S="+str(parameterS)+"_T-spectra.png")

except Exception as e: print(e)

In [ ]:
if decay in ["i","t"]:
    dff = pd.DataFrame()
    dff["freq"] = freqs
    dff["Coupling Efficiency"] = couplingE
    dff["Str"] = T_str
    dff["Bent"]=T_bent
    dff.to_csv(abspath+thetime+"S="+str(parameterS)+".csv", index=False)